In [1]:
import yaml
import sqlite3 as sql 

# Função responsavel por criar query de tabela pro sql

In [2]:
def generate_table_query(table_templete):
    table_name = list(table_templete.keys())[0]
    columns = table_templete[table_name]['columns']
    table_options = table_templete[table_name]['table_options']

    column_list = []

    for column in columns:
        col_str = f'{column['name']} {column['type']}'
        options = column.get('options')

        if options:
            if isinstance(options, list):
                col_str += ' ' + ' '.join(options)
            else:
                col_str += ' ' + options

        column_list.append(col_str)

    if table_options:
        if isinstance(table_options, list):
                column_list.extend(table_options)
        else:
            column_list.append(table_options)            

    query = f'CREATE TABLE IF NOT EXISTS {table_name} (\n '+',\n '.join(column_list)+'\n)'

    return query+';'

# Classe de debug

- Pega o nome das tabelas num arquivo yaml;
- pega os objetos em uma tabela
- Pega o tipo do objeto numa tabela

In [52]:
class WhereIsThat:
    def __init__(self, file_name: str):
        self.get_file(file_name)
        self.get_list_names()

    def get_file(self, file_name):
        with open('Templet/'+file_name, 'r') as file:
            self.content = yaml.safe_load(file)

    def get_list_names(self):
        self.list = list(self.content.keys())

    def get_names(self, table):
        columns = self.content[table]['columns']
        names = []
        for column in columns: 
            names.append(column['name'])
        return names

    def get_types(self, table):
        columns = self.content[table]['columns']
        types = []
        for column in columns: 
            types.append(column['type'])
        return types

    def what_about(self, table):
        return list(zip(self.get_names(table), self.get_types(table)))


In [53]:
test = WhereIsThat('semester.yaml')

print(test.list)
print(test.what_about('Semester'))

['Semester', 'UpComingEvents', 'User', 'Question', 'Grp', 'Participants', 'Date', 'Grade']
[('year', 'INTEGER'), ('period', 'INTEGER'), ('start', 'TEXT'), ('end', 'TEXT')]


In [5]:
with open('semester.yaml', 'r') as file:
    yaml_content = yaml.safe_load(file)

for table_name, table_content in yaml_content.items():
    print(generate_table_query({table_name: table_content}))

CREATE TABLE IF NOT EXISTS Semester (
 year INTEGER NOT NULL,
 period INTEGER NOT NULL CHECK (period IN (1, 2)),
 start TEXT,
 end TEXT,
 UNIQUE (year, period)
);
CREATE TABLE IF NOT EXISTS UpComingEvents (
 day TEXT NOT NULL,
 event TEXT
);
CREATE TABLE IF NOT EXISTS User (
 wa_id TEXT PRIMARY KEY,
 matr INTEGER DEFAULT 0,
 name TEXT DEFAULT 'UNKNOWN',
 role TEXT DEFAULT 'UNKNOWN',
 age INTEGER DEFAULT 0,
 gender TEXT DEFAULT 'UNKNOWN',
 origin TEXT DEFAULT 'UNKOWN',
 trancou INTEGER DEFAULT 0 CHECK (trancou IN (0, 1)),
 UNIQUE (wa_id, matr)
);
CREATE TABLE IF NOT EXISTS Question (
 wa_id TEXT,
 name INTEGER DEFAULT 0 CHECK (name IN (0, 1)),
 gender INTEGER DEFAULT 0 CHECK (gender IN (0, 1)),
 matr INTEGER DEFAULT 0 CHECK (matr IN (0, 1)),
 origin INTEGER DEFAULT 0 CHECK (origin IN (0, 1)),
 age INTEGER DEFAULT 0 CHECK (age IN (0, 1)),
 FOREIGN KEY (wa_id) REFERENCES User(wa_id)
);
CREATE TABLE IF NOT EXISTS Grp (
 grp_id INTEGER PRIMARY KEY AUTOINCREMENT,
 grpName TEXT,
 narrador INT

# Cria uma classe que se conecta ao DataBase 

Esta Classe:
- Realiza uma função fornecida 

Objetivo é reduzir chances de bugs e erros se o sql nunca manter uma conecção aberta por tempo de mais 

In [54]:
def try_execute(func):
    def wrapper(self, *args, **kwargs):
        try:
            conn = sql.connect('DataBase/'+self.file_name)
            self.cursor = conn.cursor()
            result = func(self, *args, **kwargs)
            self.cursor.close()
            conn.commit()
            conn.close()
            return result

        except Exception as error:
            return error

    return wrapper

In [55]:
class DbConnect:
    def __init__(self, file_name: str):
        self.file_name = file_name

    @try_execute
    def create_tables(self, yaml_file_name: str):
        with open('Templet/'+yaml_file_name, 'r') as file:
            self.yaml_content = yaml.safe_load(file)

        for table_name, table_content in self.yaml_content.items():
            query = generate_table_query({table_name: table_content})
            self.cursor.execute(query)

    @try_execute
    def insert_into(self, table_name: str, **what):
        keys = ', '.join(what.keys())
        placeholders = ', '.join(['?'] * len(what))
        values = tuple(what.values())
        query = f'''
            INSERT OR IGNORE INTO {table_name} ({keys})
            VALUES ({placeholders})
''' 
        self.cursor.execute(query, values)
    
    @try_execute
    def update_into(self, table_name: str, where: str, where_value, **to_update):
        update = ', '.join([f'{key}' for key in to_update.keys()])
        values = tuple(to_update.values()) + (where_value,)
        query = f'''
        UPDATE {table_name}
        SET {update} = ?
        WHERE {where} = ?
''' 
        self.cursor.execute(query, values)     

    @try_execute
    def fetch_one(self, table_name, what, where, where_value):
        query = f'''
        SELECT {what}
        FROM {table_name}
        WHERE {where} = ?
''' 
        self.cursor.execute(query, (where_value,))
        return self.cursor.fetchone()
    
    @try_execute
    def fetch_all(self, table_name, what):
        query = f'SELECT {what} FROM {table_name}'
        self.cursor.execute(query)
        return self.cursor.fetchall()
    

    @try_execute
    def fetch_all_abt(self, table_name, where, where_value, *what):
        
        to_fetch = ', '. join([f'{key}' for key in what ])
        query = f'''
        SELECT {to_fetch}
        FROM {table_name}
        WHERE {where} = ?
'''
        self.cursor.execute(query, (where_value,))
        return self.cursor.fetchall()
    
    
    # Funções de DEBUG
    def get_list_names(self):
        return list(self.yaml_content.keys())

    def get_names(self, table):
        columns = self.yaml_content[table]['columns']
        names = []
        for column in columns: 
            names.append(column['name'])
        return names

    def get_types(self, table):
        columns = self.yaml_content[table]['columns']
        types = []
        for column in columns: 
            types.append(column['type'])
        return types

    def what_about(self, table):
        return list(zip(self.get_names(table), self.get_types(table)))

    

In [56]:
data = DbConnect('TestFile')

In [57]:
print(data.create_tables('SemesterInfo/semester.yaml'))

None


In [58]:
data.get_list_names()

['Semester',
 'UpComingEvents',
 'User',
 'Question',
 'Grp',
 'Participants',
 'Date',
 'Grade']

In [59]:
data.what_about('Semester')

[('year', 'INTEGER'),
 ('period', 'INTEGER'),
 ('start', 'TEXT'),
 ('end', 'TEXT')]

In [60]:
data.insert_into('Semester', year=2001, period=1)

In [62]:
data.update_into('Semester', 'year', 2001, end='never')

In [63]:
data.fetch_all_abt('Semester', 'year', 2001, 'period', 'start', 'end')

[(1, None, 'never')]

- Sobre a data Base 
- Preciso corrigir certos aspectos

- Que tipo de pergunta farei ?